In [1]:
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import random
import pickle
from Layers import DenseLayer
from DNN import SimpleDNN
from math import *
import itertools
from sklearn.metrics import accuracy_score
import numpy as np

from scapy.utils import PcapReader
!pip install scapy



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [13]:
# Activation functions and their derivatives
def sigmoid(x):
    epsilon = 1e-7  # Small constant for numerical stability
    x = np.clip(x, -500, 500)  # Prevent overflow
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# Mean Squared Error Loss Function
def mse_loss(y_true, y_pred):
    return ((y_true - y_pred) ** 2).mean()


def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)


def cross_entropy_loss(y_true, y_pred):
    # Small epsilon value to prevent log(0)
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1. - epsilon)
    return -np.mean(np.sum(y_true * np.log(y_pred), axis=1))


def l1_regularization(weights, l1_lambda=0.01):
    return l1_lambda * np.sign(weights)

def l2_regularization(weights, l2_lambda=0.01):
    return l2_lambda * weights


# Map each activation function to its derivative
activation_derivatives = {
    sigmoid: sigmoid_derivative,
    relu: relu_derivative
}

In [53]:
## TEST WITH PCAP

In [4]:
import pandas as pd

# Load the datasets
attack_df = pd.read_csv('data/CTU13_Attack_Traffic.csv')
normal_df = pd.read_csv('data/CTU13_Normal_Traffic.csv')
combined_df = pd.read_csv('data/CTU13_Combined_Traffic_Corrected.csv')
# Combine the two dataframes
#combined_df = pd.concat([attack_df, normal_df], ignore_index=True)

combined_df.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Act Data Pkts,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,59086131,7,1,0,0,0,0,0.000000,0.000000,0,...,0,2987276.0,0.0,2987276,2987276,1.869962e+07,19471121.45,41116855,5999291,1
1,12452268,37,1,2408,68,68,50,65.081081,6.726310,68,...,37,0.0,0.0,0,0,0.000000e+00,0.00,0,0,1
2,118741070,5,4,170,682,45,22,34.000000,10.440307,461,...,5,2276383.0,0.0,2276383,2276383,1.161281e+08,0.00,116128125,116128125,1
3,180643,25,11,180,25790,90,0,7.200000,24.919872,5840,...,2,0.0,0.0,0,0,0.000000e+00,0.00,0,0,1
4,440,4,1,0,0,0,0,0.000000,0.000000,0,...,0,0.0,0.0,0,0,0.000000e+00,0.00,0,0,1


In [5]:
# Assuming all features except 'label' are numerical
features = combined_df.columns.difference(['Label'])
scaler = StandardScaler()
combined_df[features] = scaler.fit_transform(combined_df[features])

In [30]:
from sklearn.model_selection import train_test_split

X = combined_df.drop('Label', axis=1)
y = combined_df['Label']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)
X_train, X_test_val, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)
X_train_np = X_train.values
y_train_np = y_train.values

# Convert labels to one-hot encoding
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y_train_np.reshape(-1, 1))


/Users/smander/projects/PHD/DNN/venv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [32]:
from Layers import DenseLayer,SymbolicLayer
from DNN import SimpleDNN

# Update based on the actual number of features in your combined dataset
num_features = 57  # Replace N with the actual number

# Assuming you have a class or function to define layers, update your layers list
layers = [
    SymbolicLayer.Layer(57, 21, sigmoid, sigmoid_derivative, l2_regularization),
    DenseLayer.Layer(21, 9, sigmoid, sigmoid_derivative, l2_regularization),
    DenseLayer.Layer(9, 4, sigmoid, sigmoid_derivative, l2_regularization),
    DenseLayer.Layer(4, 2, sigmoid, sigmoid_derivative, l2_regularization),
    #DenseLayer.Layer(5, 2, sigmoid, sigmoid_derivative, l2_regularization),
]

# Assuming `SimpleDNN` and other relevant classes/functions are defined and work with the updated architecture
dnn = SimpleDNN(layers)

# Assuming `X_train` and `y_train` are correctly prepared from the combined dataset
dnn.train(X_train_np, y_onehot, epochs=100, learning_rate=0.001)

# Save your model
save_model(dnn, 'ai_network.pkl')

0
Epoch 0 - Loss: 1.1216958365559386
1
Epoch 1 - Loss: 0.1455552303953733
2
Epoch 2 - Loss: 1.2070222388787346
3
Epoch 3 - Loss: 0.7291034262280982
4
Epoch 4 - Loss: 0.5688775558467841
5
Epoch 5 - Loss: 0.8257161720596834
6
Epoch 6 - Loss: 0.6416045290645018
7
Epoch 7 - Loss: 0.6097121477811139
8
Epoch 8 - Loss: 0.8221924039515712
9
Epoch 9 - Loss: 0.8039131196001964
10
Epoch 10 - Loss: 0.6660032344263036
11
Epoch 11 - Loss: 0.6897588517116022
12
Epoch 12 - Loss: 0.6742430144845815
13
Epoch 13 - Loss: 0.7060319713201968
14
Epoch 14 - Loss: 0.6910176518248562
15
Epoch 15 - Loss: 0.7401338579264877
16
Epoch 16 - Loss: 0.7233116297888675
17
Epoch 17 - Loss: 0.7000858080067051
18
Epoch 18 - Loss: 0.679582073872629
19
Epoch 19 - Loss: 0.6877885185542054
20
Epoch 20 - Loss: 0.7162607597588655
21
Epoch 21 - Loss: 0.7404788740653016
22
Epoch 22 - Loss: 0.7734098377874907
23
Epoch 23 - Loss: 0.6847597968405119
24
Epoch 24 - Loss: 0.6301682615916455
25
Epoch 25 - Loss: 0.6582277639563479
26
Epoc

In [47]:
def predict(model, X):
    predictions = model.feedforward(X)
    if predictions.ndim > 1:  # Predictions are one-hot encoded
        return np.argmax(predictions, axis=1)
    else:  # Predictions are direct class labels
        return predictions

## LOADED MODEL LOGIC????????????????????????

# Load the model
loaded_dnn = load_model('ai_network.pkl')

scaler = StandardScaler().fit(X_train)
X_test_scaled = scaler.transform(X_test_val)  # Use transform, not fit_transform

# Ensure X_test_scaled is used for predictions
y_pred_loaded_model = predict(loaded_dnn, X_test_scaled)  # Correct variable used

# Sample data loading. Replace this with the actual path to your data file
data_path = "data/CTU13_Attack_Traffic_VAL.csv"
df = pd.read_csv(data_path)

# Separate features and labels if you have labels in your dataset
X = df.drop('Label', axis=1).values  # Assuming 'Label' is the column with labels

# If your data includes a 'Label' column and you're interested in evaluating predictions,
# you can keep labels for comparison
y = df['Label'].values if 'Label' in df else None
X_scaled = scaler.transform(X)


# Check if conversion is necessary for y_test
if y_test.ndim > 1:  # y_test is one-hot encoded
    y_test_labels = np.argmax(y_test, axis=1)
else:  # y_test is already class labels
    y_test_labels = y_test

# Calculate accuracy
accuracy_loaded_model = accuracy_score(y_test_labels, y_pred_loaded_model)
print(f"Accuracy of loaded model: {accuracy_loaded_model}")


Accuracy of loaded model: 0.960123317169902


/Users/smander/projects/PHD/DNN/venv/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [46]:
# Load the model
loaded_dnn = load_model('ai_network.pkl')

# Load new data for prediction
data_path = "data/CTU13_Attack_Traffic_VAL.csv"
df = pd.read_csv(data_path)

# Separate features; ensure they match the training feature set
X_val = df.drop(columns = ['Label'], axis=1).values

# Assuming 'Label' column exists for accuracy evaluation
y_val = df['Label'].values

# Check if conversion is necessary for y_test
if y_val.ndim > 1:  # y_test is one-hot encoded
    y_test_labels = np.argmax(y_val, axis=1)
else:  # y_test is already class labels
    y_test_labels = y_val

# Scale features using the pre-fitted scaler
X_val_scaled = scaler.transform(X_val)

# Predict using the loaded model
y_pred_val = predict(loaded_dnn, X_val_scaled)
#print(y_pred_val)
print("Predicted labels:", y_pred_val[:10])
print("Actual labels:", y_test_labels[:10])


# Calculate and print accuracy
accuracy_val = accuracy_score(y_test_labels, y_pred_val)
print(f"Accuracy of predictions on new data: {accuracy_val}")

Predicted labels: [1 1 1 1 1 1 1 1 1 1]
Actual labels: [1 1 1 1 1 1 1 1 1 1]
Accuracy of predictions on new data: 1.0


/Users/smander/projects/PHD/DNN/venv/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [48]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

def train_until_target_accuracy(X_train, y_train, X_val, y_val, layers_config, target_accuracy=0.90, epochs=1000, learning_rate=0.001):
    # Initialize model with the given layer configuration
    dnn = SimpleDNN(layers_config)

    # Scale features
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # One-hot encode labels
    encoder = OneHotEncoder(sparse=False)
    y_train_onehot = encoder.fit_transform(y_train.reshape(-1, 1))

    # Training loop
    for epoch in range(epochs):
        # Train the model for one epoch
        dnn.train(X_train_scaled, y_train_onehot, epochs=100, learning_rate=learning_rate)

        # Load new data for prediction
        data_path = "data/CTU13_Combined_Traffic_Corrected.csv"
        df = pd.read_csv(data_path)

        # Separate features; ensure they match the training feature set
        X_val = df.drop(columns = ['Label'], axis=1).values

        # Assuming 'Label' column exists for accuracy evaluation
        y_val = df['Label'].values

        # Check if conversion is necessary for y_test
        if y_val.ndim > 1:  # y_test is one-hot encoded
            y_test_labels = np.argmax(y_val, axis=1)
        else:  # y_test is already class labels
            y_test_labels = y_val

        # Scale features using the pre-fitted scaler
        X_val_scaled = scaler.transform(X_val)

        # Predict using the loaded model
        y_pred_val = predict(dnn, X_val_scaled)
        #print(y_pred_val)

        # Calculate and print accuracy
        accuracy_val = accuracy_score(y_test_labels, y_pred_val)
        print(f"Accuracy of predictions on new data: {accuracy_val}")
        # Check if target accuracy is met
        if accuracy_val >= target_accuracy:
            print(f"Target accuracy of {target_accuracy} reached at epoch {epoch}.")
            save_model(dnn, 'ai_network.pkl')
            break

    # Optionally save the model here

    return dnn  # Return the trained model

# Load your data
combined_df = pd.read_csv('data/CTU13_Combined_Traffic_Corrected.csv')
X = combined_df.drop('Label', axis=1)
y = combined_df['Label'].values

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.7, random_state=42)

# Define your neural network architecture
layers_config = [
    SymbolicLayer.Layer(57, 21, sigmoid, sigmoid_derivative, l2_regularization),
    DenseLayer.Layer(21, 9, sigmoid, sigmoid_derivative, l2_regularization),
    DenseLayer.Layer(9, 4, sigmoid, sigmoid_derivative, l2_regularization),
    DenseLayer.Layer(4, 2, sigmoid, sigmoid_derivative, l2_regularization),
]

# Train the model until it reaches the target accuracy on the validation set
trained_model = train_until_target_accuracy(X_train.values, y_train, X_val.values, y_val, layers_config, target_accuracy=0.99)


/Users/smander/projects/PHD/DNN/venv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0
Epoch 0 - Loss: 1.6172131597440338
1
Epoch 1 - Loss: 0.6901447828067705
2
Epoch 2 - Loss: 0.6230184270158821
3
Epoch 3 - Loss: 0.8009833795350843
4
Epoch 4 - Loss: 0.6596324571462691
5
Epoch 5 - Loss: 0.7246406136530249
6
Epoch 6 - Loss: 0.6688182295696422
7
Epoch 7 - Loss: 0.7047958377581806
8
Epoch 8 - Loss: 0.6739182215169971
9
Epoch 9 - Loss: 0.6942466671645828
10
Epoch 10 - Loss: 0.6774949650940052
11
Epoch 11 - Loss: 0.6874735434266512
12
Epoch 12 - Loss: 0.6803071562420945
13
Epoch 13 - Loss: 0.6824743628449967
14
Epoch 14 - Loss: 0.682636964886457
15
Epoch 15 - Loss: 0.6782229108775056
16
Epoch 16 - Loss: 0.6846244788299017
17
Epoch 17 - Loss: 0.6739354916280378
18
Epoch 18 - Loss: 0.6863946996732653
19
Epoch 19 - Loss: 0.6686351736662615
20
Epoch 20 - Loss: 0.6882012989354782
21
Epoch 21 - Loss: 0.6605106936341683
22
Epoch 22 - Loss: 0.6909907582555697
23
Epoch 23 - Loss: 0.6454226279674671
24
Epoch 24 - Loss: 0.6997191402350813
25
Epoch 25 - Loss: 0.6162540029836685
26
Epoc

KeyboardInterrupt: 

In [23]:

# Define your neural network architecture
layers = [
    DenseLayer.Layer(57, 5, sigmoid, sigmoid_derivative),
    DenseLayer.Layer(5, 2, sigmoid, sigmoid_derivative),
]

dnn = SimpleDNN(layers)

# Custom training loop with accuracy check
def custom_train(model, X_train_scaled, y_train_onehot, X_val_scaled, y_val, epochs, in_epochs, target_accuracy=0.94):
    for epoch in range(epochs):
        for in_epoch in range(in_epochs):
            # Train the model
            model.train(X_train_scaled, y_train_onehot, epochs=22, learning_rate=0.001)  # Ensure the `train` method is correctly implemented

            # Predict on validation set
            y_val_pred = predict(model, X_val_scaled)

            # Calculate validation accuracy; no need for one-hot decoding if y_val is already class labels
            val_accuracy = accuracy_score(y_val, y_val_pred)
            print(f"Epoch {epoch} In_Epoch {in_epoch} - Validation Accuracy: {val_accuracy}")

            # Check if target accuracy is met
            if val_accuracy >= target_accuracy:
                print(f"Target accuracy of {target_accuracy} reached at epoch {epoch}, in-epoch {in_epoch}. Stopping training.")
                return  # Exit the function early


combined_df = pd.read_csv('data/CTU13_Combined_Traffic_Corrected.csv')
# Assuming all features except 'label' are numerical
features = combined_df.columns.difference(['Label'])
scaler = StandardScaler()
combined_df[features] = scaler.fit_transform(combined_df[features])
from sklearn.model_selection import train_test_split

X = combined_df.drop('Label', axis=1)
y = combined_df['Label']

# Split data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.7, random_state=41)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.7, random_state=41)

# Fit the scaler on training data and transform training, validation, and test data
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)  # Prepare validation data
X_test_scaled = scaler.transform(X_test)

# Encode training and validation labels
encoder = OneHotEncoder(sparse=False)
y_train_onehot = encoder.fit_transform(y_train.values.reshape(-1, 1))
y_val_onehot = encoder.transform(y_val.values.reshape(-1, 1))  # Optionally prepare one-hot encoded validation labels if needed


# Train the model with custom training loop
# Train the model with custom training loop, including the validation set
custom_train(dnn, X_train_scaled, y_train_onehot, X_val_scaled, y_val, epochs=1000, in_epochs=33, target_accuracy=0.99)

# Save your model
save_model(dnn, 'ai_network.pkl')

# Load the model
loaded_dnn = load_model('ai_network.pkl')

# Prepare the test data
X_test_scaled = scaler.transform(X_test)  # Use transform, not fit_transform

# Predict using the loaded model
y_pred_loaded_model = predict(loaded_dnn, X_test_scaled)

# Check if conversion is necessary for y_test
if y_test.ndim > 1:
    y_test_labels = np.argmax(y_test, axis=1)
else:
    y_test_labels = y_test

# Calculate accuracy on test set
accuracy_loaded_model = accuracy_score(y_test_labels, y_pred_loaded_model)
print(f"Accuracy of loaded model on the test set: {accuracy_loaded_model}")


/Users/smander/projects/PHD/DNN/venv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0
Epoch 0 - Loss: 0.8892184825459292
Epoch 0 In_Epoch 0 - Validation Accuracy: 0.5777215451353026
0
Epoch 0 - Loss: 0.8338954973801185
Epoch 0 In_Epoch 1 - Validation Accuracy: 0.4222784548646974
0
Epoch 0 - Loss: 0.9944478510009507
Epoch 0 In_Epoch 2 - Validation Accuracy: 0.5777215451353026
0
Epoch 0 - Loss: 0.49023438739810377
Epoch 0 In_Epoch 3 - Validation Accuracy: 0.7611030778764718
0
Epoch 0 - Loss: 0.5000960922107497
Epoch 0 In_Epoch 4 - Validation Accuracy: 0.7556806444949391
0
Epoch 0 - Loss: 0.5192310706408303
Epoch 0 In_Epoch 5 - Validation Accuracy: 0.7671452179301798
0
Epoch 0 - Loss: 0.4285218570619797
Epoch 0 In_Epoch 6 - Validation Accuracy: 0.8429560008262755
0
Epoch 0 - Loss: 0.5440472229364363
Epoch 0 In_Epoch 7 - Validation Accuracy: 0.777163809130345
0
Epoch 0 - Loss: 0.3852338899849021
Epoch 0 In_Epoch 8 - Validation Accuracy: 0.8454864697376575
0
Epoch 0 - Loss: 0.34342758468592616
Epoch 0 In_Epoch 9 - Validation Accuracy: 0.8505474075604214
0
Epoch 0 - Loss: 0

KeyboardInterrupt: 

In [163]:
##TESTER TOOL FOR FINDING SPECIFIC

In [184]:
# Split the data into training, validation, and test sets

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=42)
X_train_np = X_train.values
X_val_np = X_train_val.values
y_train_np = y_train.values
y_val_np = y_train.values

# Convert labels to one-hot encoding
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y_train_np.reshape(-1, 1))

y_onehot_val = encoder.fit_transform(y_val_np.reshape(-1, 1))

# Apply scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_np)
X_val_scaled = scaler.transform(X_train_val)
X_test_scaled = scaler.transform(X_test)


#THIS IS FOR FIND BEST AMOUNT OF LAYOUTS AND SIZES AND ACTIVATION FUNCTIONS
# Define possible configurations
layer_sizes_options = [[13, 40, 20, 10, 3], [13, 30, 15, 3], [13, 50, 25, 10, 5, 3]]
activation_functions = [sigmoid]  # Add your activation functions here


def generate_layer_sizes(input_size, output_size, min_layers, max_layers, min_neurons, max_neurons):
    layer_configurations = []

    for num_layers in range(min_layers, max_layers + 1):
        for _ in range(10):  # Generate 10 random configurations per layer count
            # Randomly choose the number of neurons for each hidden layer
            hidden_layers = [random.randint(min_neurons, max_neurons) for _ in range(num_layers - 1)]
            # Always start with input size and end with output size
            layer_sizes = [input_size] + hidden_layers + [output_size]
            layer_configurations.append(layer_sizes)

    return layer_configurations


# Example usage
input_size = 57  # For the Wine dataset
output_size = 2  # Assuming one-hot encoded output with 3 classes
layer_sizes_options = generate_layer_sizes(input_size, output_size, 2, 5, 5, 100)
print(layer_sizes_options)


def generate_configurations(layer_sizes_options, activation_functions):
    for layer_sizes in layer_sizes_options:
        for activation_func in itertools.product(activation_functions, repeat=len(layer_sizes)-1):
            yield layer_sizes, activation_func


def train_and_evaluate(X_train_scaled, y_train, X_val_scaled, y_val, layer_sizes, activation_funcs, epochs,
                       learning_rate):
    # Create the network with the specified layers and activation functions
    layers = [
        DenseLayer.Layer(input_size, output_size, act_func, activation_derivatives[act_func])
        for (input_size, output_size), act_func in zip(zip(layer_sizes[:-1], layer_sizes[1:]), activation_funcs)
    ]
    dnn = SimpleDNN(layers)

    # Train the network with the training data and labels
    dnn.train(X_train_scaled, y_train, epochs, learning_rate)

    # Predict on the validation data
    y_pred = predict(dnn, X_val_scaled)

    # Calculate accuracy on the validation data
    y_val_labels = np.argmax(y_val, axis=1)
    accuracy = accuracy_score(y_val_labels, y_pred)
    return accuracy


# Main experiment loop
results = []
for layer_sizes, activation_funcs in generate_configurations(layer_sizes_options, activation_functions):
    accuracies = []
    for _ in range(10):  # Reduced number of tests for computational efficiency
        accuracy = train_and_evaluate(X_train_scaled, y_onehot, X_val_scaled, y_onehot_val, layer_sizes, activation_funcs, 20, 0.001)
        accuracies.append(accuracy)
    avg_accuracy = np.mean(accuracies)
    results.append((layer_sizes, activation_funcs, avg_accuracy))

best_config = max(results, key=lambda x: x[2])
print("Best Configuration:", best_config)

# Final evaluation on the test set with the best configuration
#best_layer_sizes, best_activation_funcs = best_config[0], best_config[1]
#final_accuracy = train_and_evaluate(X_train_scaled, y_onehot_val, X_test_scaled, y_test, best_layer_sizes, best_activation_funcs, 20, 0.001)
#print("Final Accuracy on Test Set:", final_accuracy)

/Users/smander/projects/PHD/DNN/venv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/smander/projects/PHD/DNN/venv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/smander/projects/PHD/DNN/venv/lib/python3.9/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/smander/projects/PHD/DNN/venv/lib/python3.9/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


[[57, 29, 2], [57, 36, 2], [57, 28, 2], [57, 46, 2], [57, 41, 2], [57, 7, 2], [57, 33, 2], [57, 20, 2], [57, 29, 2], [57, 28, 2], [57, 21, 8, 2], [57, 24, 14, 2], [57, 13, 14, 2], [57, 36, 20, 2], [57, 39, 21, 2], [57, 47, 20, 2], [57, 9, 5, 2], [57, 46, 18, 2], [57, 19, 26, 2], [57, 23, 26, 2], [57, 31, 9, 28, 2], [57, 33, 22, 34, 2], [57, 32, 34, 25, 2], [57, 36, 42, 36, 2], [57, 11, 5, 39, 2], [57, 19, 18, 14, 2], [57, 24, 44, 12, 2], [57, 33, 38, 29, 2], [57, 5, 48, 45, 2], [57, 39, 25, 6, 2], [57, 10, 45, 17, 31, 2], [57, 34, 6, 16, 43, 2], [57, 32, 18, 13, 5, 2], [57, 47, 12, 43, 50, 2], [57, 35, 34, 50, 46, 2], [57, 12, 20, 35, 23, 2], [57, 38, 25, 44, 41, 2], [57, 31, 40, 13, 43, 2], [57, 25, 28, 24, 50, 2], [57, 30, 23, 20, 22, 2]]
0
Epoch 0 - Loss: 6.73823906122442
10
Epoch 10 - Loss: 6.652049885199562
0
Epoch 0 - Loss: 1.4076081255760222
10
Epoch 10 - Loss: 1.7087675223064285
0
Epoch 0 - Loss: 7.119782917128114
10
Epoch 10 - Loss: 7.119474049915519
0
Epoch 0 - Loss: 5.547823

ValueError: `axis` must be fewer than the number of dimensions (1)